
# Generative Adversarial Network (GAN) on MNIST

This notebook demonstrates the implementation of a **Generative Adversarial Network (GAN)** to generate images of handwritten digits similar to those in the MNIST dataset. GANs consist of two neural networks:
- **Generator**: Generates fake images to mimic the real data.
- **Discriminator**: Distinguishes between real and fake images.

The two networks are trained simultaneously in an adversarial manner: the generator tries to fool the discriminator, while the discriminator tries to accurately classify real and fake images.

---



## 1. Loading and Preprocessing the MNIST Dataset

We use the MNIST dataset, which contains grayscale images of handwritten digits (0-9) with size 28x28 pixels. Images are normalized to the range [-1, 1] to suit the `tanh` activation function in the generator's output layer.

---



## 2. Architecture of the Generator

The **generator** takes a random noise vector (latent space) as input and produces an image that aims to mimic real data. It uses:
- Fully connected layers with increasing dimensions.
- Batch Normalization to stabilize training.
- LeakyReLU activations for intermediate layers.
- `tanh` activation for the output layer to generate pixel values in the range [-1, 1].

---



## 3. Architecture of the Discriminator

The **discriminator** takes an image as input and outputs a probability of whether the image is real or fake. It uses:
- Fully connected layers with decreasing dimensions.
- Dropout to prevent overfitting.
- LeakyReLU activations for all layers except the output.
- `sigmoid` activation for the output layer to produce probabilities.

---



## 4. Training Procedure

The GAN is trained in two steps per iteration:
1. **Discriminator Training**: The discriminator is trained to maximize its ability to distinguish between real and fake images.
2. **Generator Training**: The generator is trained to minimize the discriminator's ability to distinguish between real and fake images.

### Loss Functions
- **Binary Cross-Entropy Loss**: Used for both the generator and the discriminator.
- **Real Labels**: Set to 1 (with optional smoothing or noise).
- **Fake Labels**: Set to 0 (with optional smoothing or noise).

### Optimizers
- **Adam Optimizer** is used for both networks with different learning rates for generator and discriminator.

---



## 5. Evaluation Metrics

We use the following metrics to evaluate the performance of the GAN:
1. **Frechet Inception Distance (FID)**: Measures the distance between the distributions of real and generated data.
2. **Inception Score (IS)**: Evaluates the diversity and quality of generated data.
3. **Kernel Inception Distance (KID)**: Measures the similarity between real and generated data.

---



## 6. Visualizing Results

After each training epoch:
- Generated images are saved and visualized in a grid (8x8).
- Real images from the dataset are also visualized for comparison.

---


In [2]:
!pip install pytorch-fid
!pip install torch-fidelity


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch_fidelity import calculate_metrics
import torchvision
import os

# [EN] Гиперпараметры
latent_dim = 128      # [EN] Размер латентного пространства
img_size = 28         # [EN] Размер изображения
channels = 1          # [EN] Количество каналов
batch_size = 64       # [EN] Размер батча
lr_g = 0.0001         # [EN] Скорость обучения генератора
lr_d = 0.0004         # [EN] Скорость обучения дискриминатора
epochs = 50           # [EN] Количество эпох
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# [EN] Датасет и загрузчик данных
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # [EN] Нормализация в диапазон [-1, 1]
])

dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [4]:
# [EN] Улучшенный генератор
class ImprovedGenerator(nn.Module):
    def __init__(self, latent_dim, img_size, channels):
        super(ImprovedGenerator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, img_size * img_size * channels),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        return img.view(img.size(0), channels, img_size, img_size)

In [5]:

# [EN] Улучшенный дискриминатор
class ImprovedDiscriminator(nn.Module):
    def __init__(self, img_size, channels):
        super(ImprovedDiscriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(img_size * img_size * channels, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        return self.model(img_flat)

In [6]:
from torchvision.utils import save_image
import os

def generate_and_save_images(generator, epoch, fake_images, real_images, num_images=10):

# [EN] Создаём директорию, если её нет
    output_dir = "./generated_images"
    os.makedirs(output_dir, exist_ok=True)

    # [EN] Пути для сохранения
    fake_path = os.path.join(output_dir, f"fid_fake_epoch_{epoch}.png")
    real_path = os.path.join(output_dir, f"fid_real_epoch_{epoch}.png")

    # [EN] Сохраняем изображения
    save_image(fake_images[:64], fake_path)  # [EN] Первые 64 изображения
    save_image(real_images[:64], real_path)

    print(f"Fake images saved to: {fake_path}")
    print(f"Real images saved to: {real_path}")


generator = ImprovedGenerator(latent_dim, img_size, channels).to(device)
discriminator = ImprovedDiscriminator(img_size, channels).to(device)

optimizer_G = optim.Adam(generator.parameters(), lr=lr_g, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr_d, betas=(0.5, 0.999))

adversarial_loss = nn.BCELoss()


In [7]:
# [EN] import matplotlib.pyplot as plt
# [EN] import numpy as np

# [EN] def generate_and_save_images(generator, epoch, num_images=10):
# [EN]     # [EN] Переходим в режим оценки (evaluation mode)
# [EN]     generator.eval()

# [EN]     # [EN] Генерируем случайный шум
# [EN]     z = torch.randn(num_images, latent_dim).to(device)

# [EN]     # [EN] Генерируем изображения
# [EN]     fake_imgs = generator(z).cpu().detach()

# [EN]     # [EN] Преобразуем диапазон [-1, 1] обратно в [0, 1]
# [EN]     fake_imgs = (fake_imgs + 1) / 2

# [EN]     # [EN] Создаем фигуру
# [EN]     plt.figure(figsize=(15, 5))
# [EN]     for i in range(num_images):
# [EN]         plt.subplot(1, num_images, i + 1)
# [EN]         plt.imshow(fake_imgs[i][0], cmap='gray')
# [EN]         plt.axis('off')
# [EN]     plt.suptitle(f"Epoch {epoch}")
# [EN]     plt.show()


In [8]:
from torch_fidelity import calculate_metrics
import os
import torchvision

def calculate_all_metrics(generator, real_data_loader, epoch, latent_dim, device, num_images=64):
    generator.eval()
    fake_images = []
    real_images = []

    # [EN] Генерация фейковых изображений
    with torch.no_grad():
        z = torch.randn(num_images, latent_dim).to(device)
        fake_imgs = generator(z).cpu()
        fake_images = (fake_imgs + 1) / 2  # [EN] Перевод в диапазон [0, 1]

    # [EN] Сбор реальных изображений
    for real_imgs, _ in real_data_loader:
        real_imgs = (real_imgs + 1) / 2  # [EN] Перевод в диапазон [0, 1]
        real_images.append(real_imgs)
        if len(real_images) * real_imgs.size(0) >= num_images:
            break

    real_images = torch.cat(real_images, dim=0)[:num_images]
    fake_images = fake_images[:num_images]

    # [EN] Создание директорий для сохранения изображений
    fake_path = f"./metrics/fake_epoch_{epoch}"
    real_path = f"./metrics/real_epoch_{epoch}"
    os.makedirs(fake_path, exist_ok=True)
    os.makedirs(real_path, exist_ok=True)

    # [EN] Сохранение изображений на диск
    for i, img in enumerate(fake_images):
        torchvision.utils.save_image(img, f"{fake_path}/img_{i}.png")
    for i, img in enumerate(real_images):
        torchvision.utils.save_image(img, f"{real_path}/img_{i}.png")

    # [EN] Вычисление всех метрик через директории
    metrics = calculate_metrics(
        input1=fake_path,  # [EN] Путь к сгенерированным изображениям
        input2=real_path,  # [EN] Путь к реальным изображениям
        cuda=torch.cuda.is_available(),
        isc=True,  # [EN] Включаем Inception Score
        fid=True,  # [EN] Включаем FID
        kid=True,  # [EN] Включаем KID
        kid_subset_size=num_images  # [EN] Устанавливаем размер подвыборки для KID
    )

    return metrics


In [9]:
best_model_path =  "best.pt"
best_fid = float('inf')

for epoch in range(epochs):
    for i, (real_imgs, _) in enumerate(dataloader):
        # [EN] Обучение дискриминатора
        real_imgs = real_imgs.to(device)
        batch_size = real_imgs.size(0)
        real_labels = torch.ones(batch_size, 1).to(device) + 0.05 * torch.randn(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device) + 0.05 * torch.randn(batch_size, 1).to(device)
        real_labels = torch.clamp(real_labels, 0, 1)
        fake_labels = torch.clamp(fake_labels, 0, 1)


        optimizer_D.zero_grad()
        z = torch.randn(batch_size, latent_dim).to(device)
        fake_imgs = generator(z).detach()
        real_loss = adversarial_loss(discriminator(real_imgs), real_labels)
        fake_loss = adversarial_loss(discriminator(fake_imgs), fake_labels)
        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()

        # [EN] Обучение генератора
        optimizer_G.zero_grad()
        z = torch.randn(batch_size, latent_dim).to(device)
        fake_imgs = generator(z)
        g_loss = adversarial_loss(discriminator(fake_imgs), real_labels)
        g_loss.backward()
        optimizer_G.step()

        # [EN] Логирование каждые 100 батчей
        if i % 100 == 0:
            print(f"Epoch [{epoch+1}/{epochs}] Batch [{i}/{len(dataloader)}] "
                  f"Loss D: {d_loss.item():.4f}, Loss G: {g_loss.item():.4f}")

    # [EN] Вычисление KID
    metrics = calculate_all_metrics(generator, dataloader, epoch + 1, latent_dim, device, num_images=64)
    generate_and_save_images(generator, epoch + 1, fake_imgs, real_imgs)
    # [EN]update best model
    if metrics['frechet_inception_distance'] < best_fid:
        best_fid = metrics['frechet_inception_distance']
        torch.save(generator.state_dict(), best_model_path)
    print(f"Epoch [{epoch+1}/{epochs}] Metrics:")
    print(f"  Inception Score (IS): {metrics['inception_score_mean']:.6f}")
    print(f"  Frechet Inception Distance (FID): {metrics['frechet_inception_distance']:.6f}")
    print(f"  Kernel Inception Distance (KID): {metrics['kernel_inception_distance_mean']:.6f}")


Epoch [1/50] Batch [0/938] Loss D: 1.3975, Loss G: 0.7125
Epoch [1/50] Batch [100/938] Loss D: 1.2315, Loss G: 0.8621
Epoch [1/50] Batch [200/938] Loss D: 1.0749, Loss G: 1.1395
Epoch [1/50] Batch [300/938] Loss D: 0.9927, Loss G: 1.2133
Epoch [1/50] Batch [400/938] Loss D: 1.3794, Loss G: 1.6816
Epoch [1/50] Batch [500/938] Loss D: 1.0434, Loss G: 0.8030
Epoch [1/50] Batch [600/938] Loss D: 1.2263, Loss G: 1.9170
Epoch [1/50] Batch [700/938] Loss D: 1.1540, Loss G: 1.3059
Epoch [1/50] Batch [800/938] Loss D: 1.0582, Loss G: 1.3081
Epoch [1/50] Batch [900/938] Loss D: 1.7232, Loss G: 0.2830


Creating feature extractor "inception-v3-compat" with features ['logits_unbiased', '2048']
Extracting features from input1
Looking for samples non-recursivelty in "./metrics/fake_epoch_1" with extensions png,jpg,jpeg
Found 64 samples
/usr/local/lib/python3.10/dist-packages/torch_fidelity/datasets.py:16: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  img = torch.ByteTensor(torch.ByteStorage.from_buffer(img.tobytes())).view(height, width, 3)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation m

Fake images saved to: ./generated_images/fid_fake_epoch_1.png
Real images saved to: ./generated_images/fid_real_epoch_1.png
Epoch [1/50] Metrics:
  Inception Score (IS): 1.708873
  Frechet Inception Distance (FID): 268.646877
  Kernel Inception Distance (KID): 0.261719
Epoch [2/50] Batch [0/938] Loss D: 0.9761, Loss G: 1.0448
Epoch [2/50] Batch [100/938] Loss D: 0.9790, Loss G: 1.0781
Epoch [2/50] Batch [200/938] Loss D: 1.3495, Loss G: 1.5524
Epoch [2/50] Batch [300/938] Loss D: 0.9938, Loss G: 1.0945
Epoch [2/50] Batch [400/938] Loss D: 0.9997, Loss G: 1.2980
Epoch [2/50] Batch [500/938] Loss D: 1.1414, Loss G: 1.2169
Epoch [2/50] Batch [600/938] Loss D: 0.9020, Loss G: 1.5148
Epoch [2/50] Batch [700/938] Loss D: 1.0342, Loss G: 2.5011
Epoch [2/50] Batch [800/938] Loss D: 1.1262, Loss G: 1.0004
Epoch [2/50] Batch [900/938] Loss D: 1.0506, Loss G: 0.7494


Creating feature extractor "inception-v3-compat" with features ['logits_unbiased', '2048']
Extracting features from input1
Looking for samples non-recursivelty in "./metrics/fake_epoch_2" with extensions png,jpg,jpeg
Found 64 samples
Processing samples
Extracting features from input2
Looking for samples non-recursivelty in "./metrics/real_epoch_2" with extensions png,jpg,jpeg
Found 64 samples
Processing samples
Inception Score: 1.5958896037924253 ± 0.22066978259589742
Frechet Inception Distance: 237.98634637923692
Kernel Inception Distance: 0.24426375370176998 ± 8.98113641826504e-08


Fake images saved to: ./generated_images/fid_fake_epoch_2.png
Real images saved to: ./generated_images/fid_real_epoch_2.png
Epoch [2/50] Metrics:
  Inception Score (IS): 1.595890
  Frechet Inception Distance (FID): 237.986346
  Kernel Inception Distance (KID): 0.244264
Epoch [3/50] Batch [0/938] Loss D: 0.9728, Loss G: 1.8482
Epoch [3/50] Batch [100/938] Loss D: 0.9934, Loss G: 1.1998
Epoch [3/50] Batch [200/938] Loss D: 1.0897, Loss G: 1.4347
Epoch [3/50] Batch [300/938] Loss D: 0.9765, Loss G: 1.7250
Epoch [3/50] Batch [400/938] Loss D: 1.0389, Loss G: 1.3324
Epoch [3/50] Batch [500/938] Loss D: 0.9870, Loss G: 1.7966
Epoch [3/50] Batch [600/938] Loss D: 0.9172, Loss G: 1.9084
Epoch [3/50] Batch [700/938] Loss D: 1.1135, Loss G: 1.6912
Epoch [3/50] Batch [800/938] Loss D: 1.0560, Loss G: 0.9560
Epoch [3/50] Batch [900/938] Loss D: 1.1123, Loss G: 1.0800


...

Fake images saved to: ./generated_images/fid_fake_epoch_48.png
Real images saved to: ./generated_images/fid_real_epoch_48.png
Epoch [48/50] Metrics:
  Inception Score (IS): 1.868605
  Frechet Inception Distance (FID): 120.239835
  Kernel Inception Distance (KID): 0.054668
Epoch [49/50] Batch [0/938] Loss D: 1.3231, Loss G: 0.8714
Epoch [49/50] Batch [100/938] Loss D: 1.3558, Loss G: 0.8016
Epoch [49/50] Batch [200/938] Loss D: 1.3049, Loss G: 0.8047
Epoch [49/50] Batch [300/938] Loss D: 1.2752, Loss G: 0.7893
Epoch [49/50] Batch [400/938] Loss D: 1.3142, Loss G: 0.7538
Epoch [49/50] Batch [500/938] Loss D: 1.3126, Loss G: 0.7089
Epoch [49/50] Batch [600/938] Loss D: 1.3078, Loss G: 0.7859
Epoch [49/50] Batch [700/938] Loss D: 1.3151, Loss G: 0.7460
Epoch [49/50] Batch [800/938] Loss D: 1.3267, Loss G: 0.9248
Epoch [49/50] Batch [900/938] Loss D: 1.3467, Loss G: 0.8626


Creating feature extractor "inception-v3-compat" with features ['logits_unbiased', '2048']
Extracting features from input1
Looking for samples non-recursivelty in "./metrics/fake_epoch_49" with extensions png,jpg,jpeg
Found 64 samples
Processing samples
Extracting features from input2
Looking for samples non-recursivelty in "./metrics/real_epoch_49" with extensions png,jpg,jpeg
Found 64 samples
Processing samples
Inception Score: 1.882167532104324 ± 0.25034112329760727
Frechet Inception Distance: 121.65068950406547
Kernel Inception Distance: 0.050570225110129687 ± 2.0783362819431818e-07


Fake images saved to: ./generated_images/fid_fake_epoch_49.png
Real images saved to: ./generated_images/fid_real_epoch_49.png
Epoch [49/50] Metrics:
  Inception Score (IS): 1.882168
  Frechet Inception Distance (FID): 121.650690
  Kernel Inception Distance (KID): 0.050570
Epoch [50/50] Batch [0/938] Loss D: 1.2696, Loss G: 0.8421
Epoch [50/50] Batch [100/938] Loss D: 1.2895, Loss G: 0.7297
Epoch [50/50] Batch [200/938] Loss D: 1.2680, Loss G: 0.7597
Epoch [50/50] Batch [300/938] Loss D: 1.3785, Loss G: 0.7647
Epoch [50/50] Batch [400/938] Loss D: 1.3312, Loss G: 0.7357
Epoch [50/50] Batch [500/938] Loss D: 1.3139, Loss G: 0.7857
Epoch [50/50] Batch [600/938] Loss D: 1.3768, Loss G: 0.7935
Epoch [50/50] Batch [700/938] Loss D: 1.3255, Loss G: 0.7808
Epoch [50/50] Batch [800/938] Loss D: 1.2670, Loss G: 0.8220
Epoch [50/50] Batch [900/938] Loss D: 1.2970, Loss G: 0.8026


Creating feature extractor "inception-v3-compat" with features ['logits_unbiased', '2048']
Extracting features from input1
Looking for samples non-recursivelty in "./metrics/fake_epoch_50" with extensions png,jpg,jpeg
Found 64 samples
Processing samples
Extracting features from input2
Looking for samples non-recursivelty in "./metrics/real_epoch_50" with extensions png,jpg,jpeg
Found 64 samples
Processing samples
Inception Score: 1.9413465208333727 ± 0.153688860047775
Frechet Inception Distance: 118.4055350183103
Kernel Inception Distance: 0.05556056859001281 ± 4.166088187994252e-08


Fake images saved to: ./generated_images/fid_fake_epoch_50.png
Real images saved to: ./generated_images/fid_real_epoch_50.png
Epoch [50/50] Metrics:
  Inception Score (IS): 1.941347
  Frechet Inception Distance (FID): 118.405535
  Kernel Inception Distance (KID): 0.055561


In [10]:
best_generator = ImprovedGenerator(latent_dim, img_size, channels).to(device)
best_generator.load_state_dict(torch.load(best_model_path))
best_generator.eval()
real_imgs = next(iter(dataloader))[0].to(device)
z = torch.randn(real_imgs.size(0), latent_dim).to(device)
fake_imgs = best_generator(z)
generate_and_save_images(best_generator, "best", fake_imgs, real_imgs)

Fake images saved to: ./generated_images/fid_fake_epoch_best.png
Real images saved to: ./generated_images/fid_real_epoch_best.png


<ipython-input-10-94994b2e554f>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_generator.load_state_dict(torch.load(best_model_path))
